#Connect Google colab with your Google drive

In [ ]:
# Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#Download required files in colab

In [ ]:
downloaded = drive.CreateFile({'id':'1fhypxKeRNfcoZWfb6AOQnaUzb4wopE_6'}) # replace the id with id of file you want to access
downloaded.GetContentFile('Amazon_Unlocked_Mobile.csv')

#Checking the dataset

In [ ]:
import pandas as pd
import gensim
import numpy as np
import os
import re
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, GRU, LSTM
from keras.layers.embeddings import Embedding
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

In [ ]:
data=pd.read_csv("Amazon_Unlocked_Mobile.csv")
data.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0


In [ ]:
data['Rating']=data['Rating']-1
data.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,I feel so LUCKY to have found this used (phone...,1.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,3,"nice phone, nice up grade from my pantach revu...",0.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Very pleased,0.0
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,3,It works good but it goes slow sometimes but i...,0.0
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,3,Great phone to replace my lost phone. The only...,0.0


In [ ]:
data["Rating"].value_counts()

4    223605
0     72350
3     61392
2     31765
1     24728
Name: Rating, dtype: int64

In [ ]:
data.drop(['Product Name','Brand Name','Price','Review Votes'],axis=1,inplace=True)

In [ ]:
data.head()

,Rating,Reviews
0,4,I feel so LUCKY to have found this used (phone...
1,3,"nice phone, nice up grade from my pantach revu..."
2,4,Very pleased
3,3,It works good but it goes slow sometimes but i...
4,3,Great phone to replace my lost phone. The only...


In [ ]:
data.isnull().sum()

Rating      0
Reviews    62
dtype: int64

In [ ]:
data = data.dropna(how='any',axis=0)

In [ ]:
data.isnull().sum()

Rating     0
Reviews    0
dtype: int64

In [ ]:
data["Rating"].value_counts()

4    223580
0     72337
3     61374
2     31763
1     24724
Name: Rating, dtype: int64

In [ ]:
data.count()

Rating     413778
Reviews    413778
dtype: int64

#Preprocessing of reviews

In [ ]:
def clean_text(text):
    text = text.lower()
    
    pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    text = pattern.sub('', text)
    text = " ".join(filter(lambda x:x[0]!='@', text.split()))
    emoji = re.compile("["
                           u"\U0001F600-\U0001FFFF"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    
    text = emoji.sub(r'', text)
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)        
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text) 
    text = re.sub(r"\'ll", " will", text)  
    text = re.sub(r"\'ve", " have", text)  
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"don't", "do not", text)
    text = re.sub(r"did't", "did not", text)
    text = re.sub(r"can't", "can not", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"couldn't", "could not", text)
    text = re.sub(r"have't", "have not", text)
    text = re.sub(r"[,.\"\'!@#$%^&*(){}?/;`~:<>+=-]", "", text)
    return text

In [ ]:
import nltk

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def CleanTokenize(data):
    req_lines = list()
    lines = data["Reviews"].values.tolist()

    for line in lines:
        line = clean_text(line)
        # tokenize the text
        tokens = word_tokenize(line)
        tokens = [w.lower() for w in tokens]
        # remove puntuations
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        # remove non alphabetic characters
        words = [word for word in stripped if word.isalpha()]
        stop_words = set(stopwords.words("english"))
        # remove stop words
        stop_words.discard('not')
        words = [w for w in words if not w in stop_words]
        req_lines.append(words)    
    return req_lines

req_lines = CleanTokenize(data)
req_lines[0:5]

[['feel',
  'lucky',
  'found',
  'used',
  'phone',
  'us',
  'not',
  'used',
  'hard',
  'phone',
  'line',
  'someone',
  'upgraded',
  'sold',
  'one',
  'son',
  'liked',
  'old',
  'one',
  'finally',
  'fell',
  'apart',
  'years',
  'didnt',
  'want',
  'upgrade',
  'thank',
  'seller',
  'really',
  'appreciate',
  'honesty',
  'said',
  'used',
  'phonei',
  'recommend',
  'seller',
  'highly',
  'would'],
 ['nice',
  'phone',
  'nice',
  'grade',
  'pantach',
  'revue',
  'clean',
  'set',
  'easy',
  'set',
  'never',
  'android',
  'phone',
  'fantastic',
  'say',
  'least',
  'perfect',
  'size',
  'surfing',
  'social',
  'media',
  'great',
  'phone',
  'samsung'],
 ['pleased'],
 ['works', 'good', 'goes', 'slow', 'sometimes', 'good', 'phone', 'love'],
 ['great',
  'phone',
  'replace',
  'lost',
  'phone',
  'thing',
  'volume',
  'button',
  'not',
  'work',
  'still',
  'go',
  'settings',
  'adjust',
  'job',
  'eligible',
  'upgrade',
  'phone',
  'againthaanks']]

#Training the model

In [ ]:
test_split = 0.2
max_length = 25

tokenizer_obj = Tokenizer()
#print(tokenizer_obj.texts_to_sequences())
tokenizer_obj.fit_on_texts(req_lines)
sequences = tokenizer_obj.texts_to_sequences(req_lines)

word_index = tokenizer_obj.word_index
print("unique tokens - "+str(len(word_index)))
vocab_size = len(tokenizer_obj.word_index) + 1
print('vocab_size - '+str(vocab_size))

lines_pad = pad_sequences(sequences, maxlen=max_length, padding='post')
category =  data['Rating'].values

indices = np.arange(lines_pad.shape[0])
np.random.shuffle(indices)
lines_pad = lines_pad[indices]
category = category[indices]

n_values = np.max(category)+1
Y = np.eye(n_values)[category]

num_test_samples = int(test_split * lines_pad.shape[0])

X_train_pad = lines_pad[:-num_test_samples]
y_train = Y[:-num_test_samples]
X_test_pad = lines_pad[-num_test_samples:]
y_test = Y[-num_test_samples:]

unique tokens - 111765
vocab_size - 111766


In [ ]:
print('Shape of X_train_pad:', X_train_pad.shape)
print('Shape of y_train:', y_train.shape)

print('Shape of X_test_pad:', X_test_pad.shape)
print('Shape of y_test:', y_test.shape)

Shape of X_train_pad: (331023, 25)
Shape of y_train: (331023, 5)
Shape of X_test_pad: (82755, 25)
Shape of y_test: (82755, 5)


In [ ]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2021-02-19 19:19:48--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.82.222
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.82.222|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [ ]:
from gensim.models import KeyedVectors
EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz' # from above
word2vec_model = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

In [ ]:
embedding_dim = 300
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    if word in word2vec_model: 
        embedding_vector = word2vec_model[word]
        embedding_matrix[i] = embedding_vector

In [ ]:
embedding_layer = Embedding(len(word_index) + 1,
                            embedding_dim,
                            weights=[embedding_matrix],
                            input_length=max_length,
                            trainable=False)

In [ ]:
model_word2vec = Sequential()
model_word2vec.add(embedding_layer)
model_word2vec.add(LSTM(units=64,  dropout=0.1, recurrent_dropout=0.1))

model_word2vec.add(Dense(5, activation='softmax'))

model_word2vec.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model_word2vec.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 300)           33529800  
_________________________________________________________________
lstm (LSTM)                  (None, 64)                93440     
_________________________________________________________________
dense (Dense)                (None, 5)                 325       
Total params: 33,623,565
Trainable params: 93,765
Non-trainable params: 33,529,800
_________________________________________________________________
None


In [ ]:
checkpoint = ModelCheckpoint('model-{epoch:03d}-{val_accuracy:03f}.h5', verbose=1, monitor='val_accuracy',save_best_only=True, mode='auto')

In [ ]:
history_word2vec = model_word2vec.fit(X_train_pad, y_train, batch_size=32, epochs=22, validation_data=(X_test_pad, y_test), callbacks=[checkpoint], verbose=1)

Epoch 1/22
10345/10345 [==============================] - 511s 49ms/step - loss: 0.8908 - accuracy: 0.6716 - val_loss: 0.7812 - val_accuracy: 0.7055

Epoch 00001: val_accuracy improved from -inf to 0.70549, saving model to model-001-0.705492.h5
Epoch 2/22
10345/10345 [==============================] - 521s 50ms/step - loss: 0.7706 - accuracy: 0.7099 - val_loss: 0.7451 - val_accuracy: 0.7190

Epoch 00002: val_accuracy improved from 0.70549 to 0.71901, saving model to model-002-0.719014.h5
Epoch 3/22
10345/10345 [==============================] - 502s 49ms/step - loss: 0.7265 - accuracy: 0.7279 - val_loss: 0.7288 - val_accuracy: 0.7276

Epoch 00003: val_accuracy improved from 0.71901 to 0.72756, saving model to model-003-0.727557.h5
Epoch 4/22
10345/10345 [==============================] - 510s 49ms/step - loss: 0.6914 - accuracy: 0.7419 - val_loss: 0.6931 - val_accuracy: 0.7423

Epoch 00004: val_accuracy improved from 0.72756 to 0.74230, saving model to model-004-0.742300.h5
Epoch 5/22
